In [ ]:

import numpy as np 
import pandas as pd 



import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



In [ ]:
!unzip -u ../input/facial-keypoints-detection/test.zip
!unzip -u ../input/facial-keypoints-detection/training.zip

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df=pd.read_csv('training.csv')

In [ ]:
df.head()

In [ ]:
df_test=pd.read_csv('test.csv')
df_test.head()

In [ ]:
len(df.iloc[0]['Image'])

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
df=df.dropna()

In [ ]:
df.isnull().sum()

In [ ]:
df.iloc[0]['Image']

In [ ]:
df.reset_index(inplace = True)
df.drop(['index'],inplace=True,axis=1)

In [ ]:
df.head()

In [ ]:
df['Image'] = df['Image'].apply(lambda x: np.fromstring(x, dtype=int, sep=' '))

In [ ]:
df.iloc[0]['Image']

In [ ]:
df['Image']=df['Image'].apply(lambda x:x.reshape((96,96)))

In [ ]:
df.iloc[0]['Image']

In [ ]:
df.iloc[0]['Image'].shape

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def plot_image(df, index):
    image = plt.imshow(df['Image'][index])
    l = []
    for i in range(1,31,2):
        l.append(plt.plot(df.loc[index][i-1], df.loc[index][i], 'ro'))
        
    return image, l

In [ ]:
fig = plt.figure(figsize=(20, 20))

for i in range(20):
    ax = fig.add_subplot(5, 4, i + 1)
    plot_image(df, i)

plt.show()

In [ ]:
df.shape

In [ ]:
df_img = []
for i in range(0,df.shape[0]):
    im = df['Image'][i]
    df_img.append(im)

In [ ]:
df_img[0:5]

In [ ]:
len(df_img),df_img[0].shape

In [ ]:
df_img = np.array(df_img,dtype = 'float')
df_img = np.asarray(df_img, dtype=np.uint8).reshape(df_img.shape[0],96,96,1)

In [ ]:
df_img.shape

In [ ]:
keypoints=df.drop(['Image'],axis=1)

In [ ]:
df_keypoints = []
for i in range(0,df.shape[0]):
    points = keypoints.iloc[i,:]
    df_keypoints.append(points)

In [ ]:
df_keypoints[0]

In [ ]:
df_keypoints = np.array(df_keypoints,dtype = 'float')

In [ ]:
df_keypoints.shape

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df_img, df_keypoints, test_size=0.2, random_state=42)

In [ ]:
x_train.shape,x_test.shape

In [ ]:
y_train.shape,y_test.shape

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, AvgPool2D, BatchNormalization, Dropout, Activation, MaxPooling2D
from keras.optimizers import Adam
from keras import regularizers
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from keras.layers import Activation, Convolution2D, MaxPooling2D, BatchNormalization, Flatten, Dense, Dropout, Conv2D,MaxPool2D, ZeroPadding2D

In [ ]:
model = Sequential()

model.add(Convolution2D(32, (3,3), padding='same', use_bias=False, input_shape=(96,96,1)))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Convolution2D(32, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Convolution2D(64, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Convolution2D(64, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Convolution2D(96, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Convolution2D(96, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Convolution2D(128, (3,3),padding='same', use_bias=False))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Convolution2D(128, (3,3),padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Convolution2D(256, (3,3),padding='same',use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Convolution2D(256, (3,3),padding='same',use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Convolution2D(512, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Convolution2D(512, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())


model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(30))
model.summary()

In [ ]:
model.compile(optimizer='adam', 
              loss='mean_squared_error',
              metrics=['mae'])


In [ ]:
from keras.callbacks import EarlyStopping,ModelCheckpoint,ReduceLROnPlateau

stop = EarlyStopping(
    monitor='val_mae', 
    mode='min',
    patience=20
)

checkpoint= ModelCheckpoint(
    filepath='./',
    save_weights_only=True,
    monitor='val_mae',
    mode='min',
    save_best_only=True)

learning_rate_reduction = ReduceLROnPlateau(monitor = "val_mae",
                                            patience = 10,
                                            verbose = 1,
                                            factor = 0.3,
                                            min_lr = 0.0001)

In [ ]:
history=model.fit(x_train,y_train,epochs = 500,batch_size = 64,validation_split=0.1,callbacks=[learning_rate_reduction,stop,checkpoint])

In [ ]:
model.save("facial_keypoint2.h5")

In [ ]:
evaluate = model.evaluate(x_test, y_test)
evaluate

In [ ]:
prediction=model.predict(x_test)

In [ ]:
prediction[0]

In [ ]:
def plot_test_image(df,prediction, index):
    image = plt.imshow(df[index])
    l = []
    for i in range(1,31,2):
        l.append(plt.plot(prediction[index][i-1], prediction[index][i], 'ro'))
        
    return image, l

In [ ]:
fig = plt.figure(figsize=(20, 20))

for i in range(20):
    ax = fig.add_subplot(5, 4, i + 1)
    plot_test_image(x_test,prediction, i)

plt.show()

In [ ]:
fig = plt.figure(figsize=(20, 20))

for i in range(20):
    ax = fig.add_subplot(5, 4, i + 1)
    plot_test_image(x_test,y_test, i)

plt.show()

In [ ]:
df_test.head()

In [ ]:
df_test['Image'] = df_test['Image'].apply(lambda x: np.fromstring(x, dtype=int, sep=' ').reshape((96,96)))


In [ ]:
df_test.shape

In [ ]:
df_test.head()

In [ ]:
img_test = []
for i in range(0,1783):
    im = df_test['Image'][i]
    img_test.append(im)

In [ ]:
img_test = np.array(img_test,dtype = 'float')
img_test = np.asarray(img_test, dtype=np.uint8).reshape(img_test.shape[0],96,96,1)

In [ ]:
test_prediction=model.predict(img_test)

In [ ]:
fig = plt.figure(figsize=(20, 20))
fig.subplots_adjust(
    left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05)
for i in range(32):
    ax = fig.add_subplot(8, 4, i + 1,xticks=[],yticks=[])
    plot_test_image(img_test,test_prediction, i)

plt.show()

In [ ]:
col= list(df.columns)

In [ ]:
col.pop()

In [ ]:
col

In [ ]:
final = pd.DataFrame(test_prediction, columns = col)
final.head()

In [ ]:
lookup=pd.read_csv("../input/facial-keypoints-detection/IdLookupTable.csv")
lookup.head(10)

In [ ]:
lookup.shape

In [ ]:
print(df.columns)

In [ ]:
for i in range(lookup.shape[0]):
    lookup.Location[i] = final.loc[lookup.ImageId[i]-1][lookup.FeatureName[i]]

In [ ]:
lookup.head()

In [ ]:
lookup.drop(['ImageId','FeatureName'],axis=1,inplace=True)

In [ ]:
lookup.head()

In [ ]:
lookup.to_csv("Jatin_goyal_submission.csv",index=False)

In [ ]:
lookup.shape